In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()                        # spark session creation.
spark

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from patsy import dmatrices

sns.set_theme()

## Importing data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = "/content/drive/MyDrive/datasets/Crash_Analysis_System_(CAS)_data.csv"
df = spark.read.csv(file_path, header= True ,inferSchema = True) 

In [ ]:
df.count() , len(df.columns)

(776878, 72)

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+---+---+--------+-------------+----------+-------+------+---+---------------+---------+-------------------------+------------------+--------------+--------------+-----------------+-------------+------------------+---------+------+------------------------+-----+----------+-----+--------+---------+-------+---------------+------------+----+-----+-----------+----------------+-----+----------+-------------+---------------------+-----------+----------------+--------+-------------+----------+-----------+----------+------+-------------+--------+-----------+---------+---------+------------------+-----------+----------+-----------+-----------+---+----+-------------------+-----+-------+--------------+-------------+-----------+-----+----+-----+------------------+-----+------------+-------+----------+--------+--------+
|  X|  Y|OBJECTID|advisorySpeed|areaUnitID|bicycle|bridge|bus|carStationWagon|cliffBank|crashDirectionDescription|crashFinancialYear|crashLocation1|crashLocation2|crashRoadSideRoad|

In [ ]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+---+--------+-------------+----------+-------+------+---+---------------+---------+-------------------------+------------------+--------------+--------------+-----------------+-------------+------------------+---------+------+------------------------+------+----------+------+--------+---------+-------+---------------+------------+------+-----+-----------+----------------+-----+----------+-------------+---------------------+-----------+----------------+--------+-------------+----------+-----------+----------+------+-------------+--------+-----------+---------+---------+------------------+-----------+----------+-----------+-----------+---+----+-------------------+-----+-------+--------------+-------------+-----------+------+------+-----+------------------+-----+------------+-------+----------+--------+--------+
|  X|  Y|OBJECTID|advisorySpeed|areaUnitID|bicycle|bridge|bus|carStationWagon|cliffBank|crashDirectionDescription|crashFinancialYear|crashLocation1|crashLocation2|crashRoadSi

#### preprocessing in pyspark

In [ ]:
df = df.na.fill(np.nan, subset=None)

In [ ]:
df = df.drop("OBJECTID","advisorySpeed","crashRoadSideRoad","tlaId","meshblockId","intersection","areaUnitID","temporarySpeedLimit","pedestrian","holiday","weatherB")
df = df.distinct()

In [ ]:
df.count(),len(df.columns)

(776125, 61)

In [ ]:
df = df.withColumn('fatalCount',when(df.crashSeverity == "Fatal Crash", 1).otherwise(0))
df = df.withColumn('minorInjuryCount',when(df.crashSeverity == "Minor Crash", 1).otherwise(0))
df = df.withColumn('seriousInjuryCount',when(df.crashSeverity == "Serious Crash", 1).otherwise(0))

In [ ]:
cols = ['bicycle','bus','carStationWagon','moped','motorcycle','otherVehicleType','schoolBus','suv','taxi','truck','unknownVehicleType','vanOrUtility','bridge','cliffBank','debris','ditch','fence','truck','guardRail','houseOrBuilding','kerb','NumberOfLanes','objectThrownOrDropped','otherObject','overBank','parkedVehicle','phoneBoxEtc','postOrPole','roadworks','slipOrFlood','speedLimit','strayAnimal','train','tree','vehicle','trafficIsland','trafficSign','waterRiver']
df = df.fillna(0, subset=cols)

In [ ]:
datasets = df.randomSplit([0.7, 0.2, 0.1])

In [ ]:
datasets[0].count(),datasets[1].count(),datasets[2].count()

(543011, 155775, 77339)

In [ ]:
traindata = datasets[0]
testdata = datasets[1]
validata = datasets[2]